In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np

In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Ilostat")

In [3]:
df = pd.read_csv("ilostat_paises.csv")

In [4]:
df.head(10)

,Reference area,Selecionado,Source,Sex,Time,Total,Public,Private,Not elsewhere classified,Unnamed: 9
0,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2012,6420.0,680.0,5740.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
1,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2012,5358.0,589.0,4769.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
2,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2012,1061.0,91.0,971.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
3,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2014,7003.0,722.0,6281.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
4,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2014,5319.0,649.0,4670.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
5,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2014,1684.0,73.0,1611.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
6,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2017,6397.0,649.0,5748.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
7,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2017,5018.0,584.0,4435.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
8,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2017,1378.0,65.0,1313.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
9,Afghanistan,NaN,LFS - Labour Force Survey,Total,2020,6079.0,532.0,5547.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...


In [7]:
df.columns

Index(['Reference area', 'Selecionado', 'Source', 'Sex', 'Time', 'Total',
       'Public', 'Private', 'Not elsewhere classified', 'Unnamed: 9'],
      dtype='object')

In [5]:
## Passar para long

df_long = df.melt(id_vars =['Reference area', 'Source','Time','Sex'], 
        value_vars =['Public', 'Private'], 
        var_name ='Setor', 
        value_name ='Total')
df_long

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,HIES - Living Conditions Survey,2012,Total,Public,680.0
1,Afghanistan,HIES - Living Conditions Survey,2012,Male,Public,589.0
2,Afghanistan,HIES - Living Conditions Survey,2012,Female,Public,91.0
3,Afghanistan,HIES - Living Conditions Survey,2014,Total,Public,722.0
4,Afghanistan,HIES - Living Conditions Survey,2014,Male,Public,649.0
...,...,...,...,...,...,...
6347,Zimbabwe,LFS - Labour Force Survey,2014,Male,Private,2888.0
6348,Zimbabwe,LFS - Labour Force Survey,2014,Female,Private,3034.0
6349,Zimbabwe,LFS - Labour Force Survey,2019,Total,Private,4587.0
6350,Zimbabwe,LFS - Labour Force Survey,2019,Male,Private,2271.0


In [6]:
## fazer o total

df = df_long[df_long['Sex']=="Total"]


In [7]:
df

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,HIES - Living Conditions Survey,2012,Total,Public,680.0
3,Afghanistan,HIES - Living Conditions Survey,2014,Total,Public,722.0
6,Afghanistan,HIES - Living Conditions Survey,2017,Total,Public,649.0
9,Afghanistan,LFS - Labour Force Survey,2020,Total,Public,532.0
12,Afghanistan,LFS - Labour Force Survey,2021,Total,Public,643.0
...,...,...,...,...,...,...
6337,Zambia,LFS - Labour Force Survey,2019,Total,Private,5101.0
6340,Zambia,LFS - Labour Force Survey,2020,Total,Private,5249.0
6343,Zimbabwe,LFS - Labour Force Survey,2011,Total,Private,5057.0
6346,Zimbabwe,LFS - Labour Force Survey,2014,Total,Private,5922.0


In [8]:
df.columns

Index(['Reference area', 'Source', 'Time', 'Sex', 'Setor', 'Total'], dtype='object')

In [11]:
x=df[['Reference area', 'Source', 'Time', 'Setor', 'Total']].pivot_table(index=['Reference area','Time' ], values='Total',aggfunc=np.sum)
y = pd.DataFrame(x)
total = y.reset_index()

In [13]:
df= df[['Reference area', 'Source', 'Time', 'Setor', 'Total']]
df= df.rename(columns={"Total":"quantidade_vinculos"})

In [17]:
df1= df.merge(total,right_on=['Reference area','Time'],left_on=['Reference area','Time'])

In [19]:
df1['prop'] = df1['quantidade_vinculos']/df1['Total']

In [26]:
df1[df1['Reference area']=="United Kingdom"]

,Reference area,Source,Time,Setor,quantidade_vinculos,Total,prop
2018,United Kingdom,LFS - Labour Force Survey,2010,Public,7264.0,28720.0,0.252925
2019,United Kingdom,LFS - Labour Force Survey,2010,Private,21456.0,28720.0,0.747075
2020,United Kingdom,LFS - Labour Force Survey,2011,Public,7125.0,28995.0,0.245732
2021,United Kingdom,LFS - Labour Force Survey,2011,Private,21870.0,28995.0,0.754268
2022,United Kingdom,LFS - Labour Force Survey,2012,Public,6846.0,29363.0,0.233151
2023,United Kingdom,LFS - Labour Force Survey,2012,Private,22517.0,29363.0,0.766849
2024,United Kingdom,LFS - Labour Force Survey,2013,Public,6965.0,29603.0,0.235280
2025,United Kingdom,LFS - Labour Force Survey,2013,Private,22638.0,29603.0,0.764720
2026,United Kingdom,LFS - Labour Force Survey,2014,Public,6936.0,30433.0,0.227910
2027,United Kingdom,LFS - Labour Force Survey,2014,Private,23497.0,30433.0,0.772090


In [ ]:
#Japan
#United Kingdom: 2038


In [13]:
## Subindo para GBQ

client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao') # verificar a categoria do dado

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            52 non-null     object
 1   fonte_pesquisa  52 non-null     object
 2   genero          52 non-null     object
 3   quantidade      52 non-null     int64 
 4   setor           52 non-null     object
 5   ano_pesquisa    52 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.6+ KB


In [9]:
df=df.rename(columns={'País':'pais'})

In [11]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('genero','STRING',description='Gênero da pessoa'),
bigquery.SchemaField('quantidade','INTEGER',description='total de pessoas que trabalham'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
 bigquery.SchemaField('ano_pesquisa','Float',description='Ano de coleta da informação'),
 
 ]

In [15]:
table_ref = dataset_ref.table('ILOSTAT_paises_selecionados_genero') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=5a6f682d-0659-40e9-abae-08c21fb4e6ab>